In [12]:
import pandas as pd
from pathlib import Path

PATH_TO_DATASET = "../../dataset/"

datasets = {}
for path in Path(PATH_TO_DATASET).rglob('*.csv'):
    print(f'Processing {path}')
    datasets[path.stem] = pd.read_csv(path)

Processing ../../dataset/mission_progress.csv
Processing ../../dataset/meta_info.csv
Processing ../../dataset/sended_messages.csv
Processing ../../dataset/messages_from_agents.csv
Processing ../../dataset/target_in_fov.csv
Processing ../../dataset/global_strategy.csv
Processing ../../dataset/positions.csv
Processing ../../dataset/latest_agents_information.csv
Processing ../../dataset/local_strategy.csv


In [13]:
sended_messages_df = datasets['sended_messages']
messages_from_agents_df = datasets['messages_from_agents']
mission_progress_df = datasets['mission_progress']
target_in_fov_df = datasets['target_in_fov']
global_strategy_df = datasets['global_strategy']
positions_df = datasets['positions']
latest_agents_information_df = datasets['latest_agents_information']
local_strategy_df = datasets['local_strategy']
meta_info_df = datasets['meta_info']

In [14]:
class_agent_mapping = {
    "commander": 1,
    "scout": 2,
    "rescuer": 3,
    }


dataset_split = {
    "train": [
        1,
        2,
    ],
    "val": [
        3,
        4,
    ]
}

In [15]:
meta_info_df

,id,agent_id,role,mission
0,4,1,scout,Find the position of targets and send coordina...
1,4,2,rescuer,"Based on the coordinates provided by scout, sc..."
2,4,3,scout_commander,Coordinate scouts and rescuers to find and sca...
3,3,1,scout,Find the position of targets and send coordina...
4,3,2,rescuer,"Based on the coordinates provided by scout, sc..."
5,3,3,scout_commander,Coordinate scouts and rescuers to find and sca...
6,1,1,scout,Find the position of targets and send coordina...
7,1,2,rescuer,"Based on the coordinates provided by scout, sc..."
8,1,3,scout_commander,Coordinate scouts and rescuers to find and sca...
9,2,1,scout,Find the position of targets and send coordina...


In [16]:
train_dataset = []
test_dataset = []

SKIP_FIRST_N_TIMESTAMPS = 1

# We are iterating over the sended messages
for _, sample_group in sended_messages_df.groupby('id'):
    meta_info = meta_info_df[meta_info_df["id"] == sample_group["id"].iloc[0]]
    for _, agent in sample_group.groupby("ego_id"):
        all_timestamps = agent['timestamp'].unique()
        for timestamp in all_timestamps:
            print("Preparing sample for agent:", agent["id"].iloc[0], "at timestamp:", timestamp)
            sample = {}
            
            sample["id"] = agent["id"].iloc[0]
            sample["ego_id"] = agent["ego_id"].iloc[0]
            sample["current_timestamp"] = timestamp
            sample["meta_info"] = meta_info
            
            sample["sended_messages"] = agent[agent["timestamp"] <= timestamp]
            
            # Messages from agents
            sample["messages_from_agent"] = messages_from_agents_df[
                (messages_from_agents_df["id"] == sample_group["id"].iloc[0]) &
                (messages_from_agents_df["ego_id"] == agent["ego_id"].iloc[0]) &
                # Only messages before the current timestamp, because messages for the CURRENT timestamp are not available yet,
                # that messages are kind of "response" to the current agent's messages, so, they are not available yet
                (messages_from_agents_df["timestamp"] < timestamp) 
            ]
            
            # Mission progress
            sample["mission_progress"] = mission_progress_df[
                (mission_progress_df["id"] == sample_group["id"].iloc[0]) &
                (mission_progress_df["ego_id"] == agent["ego_id"].iloc[0]) &
                (mission_progress_df["timestamp"] <= timestamp)
            ]
            
            # Target in FOV
            sample["target_in_fov"] = target_in_fov_df[
                (target_in_fov_df["id"] == sample_group["id"].iloc[0]) &
                (target_in_fov_df["ego_id"] == agent["ego_id"].iloc[0]) &
                (target_in_fov_df["timestamp"] <= timestamp)
            ]
            
            # Global strategy
            sample["global_strategy"] = global_strategy_df[
                (global_strategy_df["id"] == sample_group["id"].iloc[0]) &
                (global_strategy_df["ego_id"] == agent["ego_id"].iloc[0]) &
                (global_strategy_df["timestamp"] <= timestamp)
            ].iloc[-1].to_frame().T
            
            # Positions
            sample["positions"] = positions_df[
                (positions_df["id"] == sample_group["id"].iloc[0]) &
                (positions_df["ego_id"] == agent["ego_id"].iloc[0]) &
                (positions_df["timestamp"] <= timestamp)
            ]
            
            # Latest agents information
            sample["latest_agent_information"] = latest_agents_information_df[
                (latest_agents_information_df["id"] == sample_group["id"].iloc[0]) &
                (latest_agents_information_df["ego_id"] == agent["ego_id"].iloc[0]) &
                (latest_agents_information_df["timestamp"] <= timestamp)
            ]
            
            # Local strategy
            sample["local_strategy"] = local_strategy_df[
                (local_strategy_df["id"] == sample_group["id"].iloc[0]) &
                (local_strategy_df["ego_id"] == agent["ego_id"].iloc[0]) &
                (local_strategy_df["timestamp"] == timestamp)
            ]
            
            if sample_group["id"].iloc[0] in dataset_split["train"]:
                print("Adding sample with id", sample["id"], "to train dataset")
                train_dataset.append(sample)
            else:
                test_dataset.append(sample)

print(f"Total samples in train dataset: {len(train_dataset)}")
print(f"Total samples in test dataset: {len(test_dataset)}")

Preparing sample for agent: 1 at timestamp: 10
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 40
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 70
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 85
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 115
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 135
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 185
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 262
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 10
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 40
Adding sample with id 1 to train dataset
Preparing sample for agent: 1 at timestamp: 70
Adding sample with id 1 to train dataset
Preparing sample for agent: 

In [17]:
import pickle

with open('train_dataset.pkl', 'wb') as f:
    pickle.dump(train_dataset, f)
    
with open('val_dataset.pkl', 'wb') as f:
    pickle.dump(test_dataset, f)

In [1]:
import pickle

with open('train_dataset.pkl', 'rb') as f:
    train_dataset = pickle.load(f)

In [2]:
train_dataset[0]["meta_info"]

,id,ego_id,role,mission
6,1,1,scout,Find the position of targets and send coordina...
7,1,2,rescuer,"Based on the coordinates provided by scout, sc..."
8,1,3,scout_commander,Coordinate scouts and rescuers to find and sca...


### Data visualization

In [1]:
"""
Utility for rendering key–value dictionaries as decorated ASCII/ANSI tables.

Example
-------
from refactored_draw_block import draw_block

print(draw_block(
    {
        "send_message_to_receiver_id": 2,
        "sended_message": (
            "Rescuer 2 — confirm: is final target at (81, 15) secured and extraction "
            "complete? Scout 1 — maintain return trajectory. Confirm once base is "
            "reached. All units — hold position pending final verification."
        ),
        "sended_message_type": "order",
    },
    title="sended_messages",
))
"""

from __future__ import annotations

import re
import textwrap
from typing import Dict, Union


class Ansi:
    """Simple namespace for ANSI color codes."""

    RESET = "\033[0m"
    BOLD = "\033[1m"

    class FG:
        BLUE = "\033[94m"
        GREEN = "\033[92m"
        CYAN = "\033[96m"


_ANSI_RE = re.compile(r"\033\[[0-9;]*m")


def _visible_len(text: str) -> int:
    """Length of *text* without counting ANSI escape sequences."""
    return len(_ANSI_RE.sub("", text))


def _clr(text: str, *codes: str, enable: bool) -> str:
    """Return *text* wrapped in ANSI *codes* if *enable* is True."""
    return f"{''.join(codes)}{text}{Ansi.RESET}" if enable else text


def _wrap_value(value: str, first_line_width: int, cont_width: int) -> list[str]:
    """Word-wrap *value* so that the first line fits *first_line_width* and later lines
    fit *cont_width* (both exclusive of indentation handled by the caller)."""

    if first_line_width <= 0 or cont_width <= 0:
        return [value]  # nothing we can do sensibly

    words = value.split()
    if not words:
        return [""]

    lines: list[str] = []
    current = words[0]

    def flush():
        if lines or len(current) <= first_line_width:
            lines.append(current)
        else:  # first word itself does not fit – hard slice
            lines.append(current[: first_line_width])
            remainder = current[first_line_width :].lstrip()
            if remainder:
                lines.extend(textwrap.wrap(remainder, cont_width))

    for word in words[1:]:
        limit = first_line_width if not lines else cont_width
        if len(current) + 1 + len(word) > limit:
            flush()
            current = word
        else:
            current += " " + word
    flush()
    return lines


def draw_block(
    fields: Dict[str, Union[str, int]],
    *,
    title: str = "sended_messages",
    width: int = 50,
    colorize: bool = True,
) -> str:
    """Return a pretty-printed ASCII/ANSI block of *fields*.

    Parameters
    ----------
    fields
        Mapping of field names to their values.
    title
        Header caption. Underscores are replaced with spaces.
    width
        Total width including borders.
    colorize
        Toggle ANSI color output.
    """
    if width < 10:
        raise ValueError("`width` must be at least 10 characters.")

    inner = width - 4  # usable width between "| " and " |"

    header = f" {title.replace('_', ' ').capitalize()} ".center(width, "-")
    header = _clr(header, Ansi.BOLD, Ansi.FG.CYAN, enable=colorize)
    footer = _clr("-" * width, Ansi.FG.CYAN, enable=colorize)

    body: list[str] = []

    for key, val in fields.items():
        key_plain = str(key)
        val_plain = str(val)

        indent_len = len(key_plain) + 2  # space reserved for "key: " in first line
        first_avail = inner - indent_len
        cont_avail = inner - indent_len

        wrapped = _wrap_value(val_plain, first_avail, cont_avail)
        indent = " " * indent_len

        for i, part in enumerate(wrapped):
            if i == 0:
                raw_line = f"{key_plain}: {part}"
                colored_line = (
                    _clr(key_plain, Ansi.BOLD, Ansi.FG.BLUE, enable=colorize)
                    + ": "
                    + _clr(part, Ansi.FG.GREEN, enable=colorize)
                    if colorize
                    else raw_line
                )
            else:
                raw_line = indent + part
                colored_line = (
                    indent + _clr(part, Ansi.FG.GREEN, enable=colorize)
                    if colorize
                    else raw_line
                )

            pad = inner - _visible_len(colored_line)
            if pad < 0:
                colored_line = colored_line[: pad]
                pad = 0
            body.append(f"| {colored_line}{' ' * pad} |")

    return "\n".join([header, *body, footer])


In [2]:
from collections import defaultdict
from enum import Enum
from itertools import zip_longest

class TableColumnNames(Enum):
    CURRENT_TIMESTAMP = "current_timestamp"
    SENDED_MESSAGES = "sended_messages"
    MESSAGES_FROM_AGENT = "messages_from_agent"
    MISSION_PROGRESS = "mission_progress"
    TARGET_IN_FOV = "target_in_fov"
    GLOBAL_STRATEGY = "global_strategy"
    POSITIONS = "positions"
    LATEST_AGENT_INFORMATION = "latest_agent_information"
    LOCAL_STRATEGY = "local_strategy"


class Table:
    def __init__(self):
        self._max_col_height_for_current_timestamp = 0
        self._all_timestamps = set()
        self._max_col_height = {}
        self._res_table = ""
        self._k2col_name = {
            TableColumnNames.CURRENT_TIMESTAMP: "Timestamp",
            TableColumnNames.SENDED_MESSAGES: "Sended Messages",
            TableColumnNames.MESSAGES_FROM_AGENT: "Messages From Agents",
            TableColumnNames.MISSION_PROGRESS: "Mission Progress",
            TableColumnNames.TARGET_IN_FOV: "Target In FOV",
            TableColumnNames.GLOBAL_STRATEGY: "Global Strategy",
            TableColumnNames.POSITIONS: "Positions",
            TableColumnNames.LATEST_AGENT_INFORMATION: "Latest Agents Information",
            TableColumnNames.LOCAL_STRATEGY: "Local Strategy"
        }
        self._table = {
            TableColumnNames.SENDED_MESSAGES: defaultdict(self.default_list_int),
            TableColumnNames.MESSAGES_FROM_AGENT: defaultdict(self.default_list_int),
            TableColumnNames.MISSION_PROGRESS: defaultdict(self.default_list_int),
            TableColumnNames.TARGET_IN_FOV: defaultdict(self.default_list_int),
            TableColumnNames.GLOBAL_STRATEGY: defaultdict(self.default_list_int),
            TableColumnNames.POSITIONS: defaultdict(self.default_list_int),
            TableColumnNames.LATEST_AGENT_INFORMATION: defaultdict(self.default_list_int),
            TableColumnNames.LOCAL_STRATEGY: defaultdict(self.default_list_int),
        }
        self.col_order = [
            TableColumnNames.CURRENT_TIMESTAMP,
            TableColumnNames.SENDED_MESSAGES,
            TableColumnNames.MESSAGES_FROM_AGENT,
            TableColumnNames.MISSION_PROGRESS,
            TableColumnNames.TARGET_IN_FOV,
            TableColumnNames.GLOBAL_STRATEGY,
            TableColumnNames.POSITIONS,
            TableColumnNames.LATEST_AGENT_INFORMATION,
            TableColumnNames.LOCAL_STRATEGY,
        ]

    def clean(self):
        for column in self._table.values():
            column.clear()
        
        self._res_table = ""

    def default_list_int(self):
        return ([], 0)

    def add_timestamp(self, timestamp: int): 
        self._all_timestamps.add(timestamp)

    def add_text_block(self, timestamp: int, title: str, text_block: str):
        col_list, total_height = self._table[title][timestamp]
        col_list.append(text_block)
        total_height += text_block.count("\n") + 1
        self._table[title][timestamp] = (col_list, total_height)

    def _calculate_max_height_for_each_timestamp(self) -> str:
        for timestamp in self._all_timestamps:
            cur_timestamp_height = 0    
            for _, v in self._table.items():
                timestamps = list(v.keys())
                if timestamp in timestamps:
                    _, height = v[timestamp]
                    cur_timestamp_height = max(cur_timestamp_height, height)
            self._max_col_height[timestamp] = cur_timestamp_height


    def merge_blocks(self, left: str, right: str, sep: str = "  ") -> str:
        """Склеить два многострочных текста горизонтально,
        выравнивая по самой длинной строке из ОБОИХ блоков."""
        l_lines = left.splitlines()
        r_lines = right.splitlines()

        # ─── главное изменение: учитываем обе стороны ───
        width = max(
            (len(s) for s in l_lines + r_lines),   # ← объединённый список
            default=0
        ) + len(sep)

        max_len = max(len(l_lines), len(r_lines))
        l_lines.extend([""] * (max_len - len(l_lines)))
        r_lines.extend([""] * (max_len - len(r_lines)))

        return "\n".join(l.ljust(width) + r for l, r in zip(l_lines, r_lines))


    # ------------------ новый build ------------------
    def build(self, sep: str = "  ") -> str:
        """Собрать таблицу с *жёсткой* шириной колонок."""
        # 0. подготовка
        self._calculate_max_height_for_each_timestamp()

        # 1. ширина каждой колонки ---------------------------------
        COL_WIDTH: dict[TableColumnNames, int] = {
            TableColumnNames.CURRENT_TIMESTAMP:
                max(len(f"|{ts}|") for ts in self._all_timestamps),   #  |40|
        }
        FIXED_BLOCK_WIDTH = 50        # ← ваши блоки
        for col in self._table:
            COL_WIDTH[col] = FIXED_BLOCK_WIDTH

        # 3. формируем все строки сразу ---------------------------
        lines: list[str] = []
        for ts in sorted(self._all_timestamps):
            height = self._max_col_height[ts]              # сколько «доп» строк

            # --- собираем «колонки» как списки строк одинаковой длины
            col_buffers: list[list[str]] = []
            for col in self.col_order:
                width = COL_WIDTH[col]

                if col is TableColumnNames.CURRENT_TIMESTAMP:
                    buf = [f"|{ts}|".ljust(width)]
                    buf.extend(["|  |".ljust(width)] * height)

                else:
                    # строки блока или пустые
                    if ts in self._table[col]:
                        blocks, _ = self._table[col][ts]
                        buf = []
                        for block in blocks:
                            buf.extend(s.ljust(width) for s in block.splitlines())
                    else:
                        buf = []

                    # довыровняем высоту (height+1 строк включая первую)
                    buf.extend(["|".ljust(width)] * (height + 1 - len(buf)))

                col_buffers.append(buf)

            # --- горизонтально склеиваем всё сразу
            for row_items in zip_longest(*col_buffers, fillvalue=" " * FIXED_BLOCK_WIDTH):
                lines.append(sep.join(row_items))

        self._res_table = "\n".join(lines)
        return self._res_table


In [4]:
import pickle

with open('train_dataset.pkl', 'rb') as f:
    train_dataset = pickle.load(f)

with open('val_dataset.pkl', 'rb') as f:
    val_dataset = pickle.load(f)

dataset = val_dataset


In [5]:
len(dataset)

38

In [10]:
len(dataset)

38

In [12]:
# columns_to_remove = ["id", "ego_id"]
columns_to_remove = []

table = Table()
generate_for = ["scout"] # "rescuer", "scout_commander"

for sample_id, sample in enumerate(dataset):
    # sample = dataset[sample_id]
    role2id = sample["meta_info"].set_index('agent_id')['role'].to_dict()
    ego_id = sample["ego_id"]
    
    role = role2id[ego_id]
    
    if role not in generate_for:
        continue

    for _key in table.col_order:
        if _key == TableColumnNames.CURRENT_TIMESTAMP:
            continue
        data_table = (sample[_key.value].to_dict(orient='records'))
        for row in data_table:
            timestamp = row["timestamp"]

            row = {k: v for k, v in row.items() if k not in columns_to_remove}
            sample_text_block = draw_block(fields=row, title=_key.value, colorize=False)
            table.add_text_block(timestamp, _key, sample_text_block)
            table.add_timestamp(timestamp)

    table.build()
    
    with open(f"sample_{sample_id}.txt", "w", encoding="utf-8") as file:
        file.write(table._res_table)
        
    table.clean()
    

In [61]:
train_dataset[1]["sended_messages"]

,id,timestamp,ego_id,send_message_to_receiver_id,sended_message,sended_message_type
57,1,10,1,2,No targets detected within current visual range.,info
58,1,10,1,3,No targets detected within current visual range.,info
59,1,40,1,2,"Two targets visually confirmed at (63, 44) and...",info
60,1,40,1,3,"Two targets visually confirmed at (63, 44) and...",info


### Dataset preparation for language models

In [16]:
import pickle
with open('train_dataset.pkl', 'rb') as f:
    train_dataset = pickle.load(f)
    
with open('val_dataset.pkl', 'rb') as f:
    val_dataset = pickle.load(f)
    

- Each sample contains following data:
    
    - **ego_id** (int) 
        - drone's ID
    - **current_timestamp** (int) 
        - Max timestamp in the sample 
    - **meta_info** (pd.DataFrame) 
        - Dataframe contains `ego_id`, `role` and `mission`
    - **sended_messages** (pd.DataFrame) 
        - Dataframe contains `sended_messages`, `sended_message_type`, `receiver_id`, `timestamps`, `ego_id` 
    - **messages_from_agent** (pd.DataFrame) 
        - Dataframe contains `timestamp`, `ego_id`, `sender id`, `message`, `message type`
    - **mission_progress** (pd.DataFrame) 
        - Dataframe contains `timestamp`, `ego_id`, `mission_progress`
    - **target_in_fov** (pd.DataFrame) 
        - Dataframe contains `timestamp`, `ego_id`, `target_pos_x`, `target_pos_y`
    - **global_strategy** (pd.DataFrame) 
        - Dataframe contains `timestamp`, `ego_id`, `global_strategy`
    - **positions** (pd.DataFrame) 
        - Dataframe contains `timestamp`,  `ego_id`, `ego_id_pos_x`, `ego_id_pos_y`, 
    - **latest_agent_information** (pd.DataFrame) 
        - Dataframe contains `timestamp`,  `ego_id`, `latest_info_agent_id`, `latest_info_agent_timestamp`, `latest_info_agent_pos_x`, `latest_info_agent_pos_y`
    - **local_strategy** (pd.DataFrame) 
        - Dataframe contains `timestamp`,  `ego_id`, `local_strategy`

As a grand truth the message on the latest timestamp is used

#### Tokens

| Токен               | Смысл                                                        | Пример строки после конверсии          |
| ------------------- | ------------------------------------------------------------ | -------------------------------------- |
| `<T+Δ>`             | «Сколько тиков от последней строки» (Δ = 0…7, 8+ → `<T+8>`). | `<T+0> <POS> 58 84`                    |
| `<SND>`, `<RCV>`    | наше отправленное / полученное сообщение                     | `<RCV> from 3 order Scout 1—redeploy…` |
| `<INFO>`, `<ORDER>` | тип сообщения                                                | —                                      |
| `<EGO_POS>`, `<POS>`, `<TGT>`    | собственная позиция / цель в поле зрения                     | `<TGT> 63 44`                          |
| `<AG#n>`            | ссылка на агента `n` в тексте сообщения                      | `Scout <AG#1> — redeploy…`             |
| `<TOME>`            | В полученных сообщениях: наш id или роль.  | `<T+0> <RCV> <TO_ME> from <AG#3> <ORDER> redeploy northward` |
|`<AGSTATE> `|summarizing-блок перед историей | `<T+0> <AGSTATE> <AG#2> <POS> 58 84` <br> `<T+5> <AGSTATE> <AG#3> <POS> 20 33 ` |
|`<PRGS>`| mission_progress | `<PRGS>` Recon unit... |
|`GLOB_STG`|global strategy|`<GLOB_STG>` scout targets |
|`LOCAL_STG`| local strategy | `<LOCAL_STG>` Repositioning northward |
|`<SCOUT>`, `<COMMANDER>`, `<RESCUER>`| Roles | `<SCOUT>` `<AG#1>` You need to scout |


**Список доступных тиков**
<T+0>, <T+1>, <T+2>, <T+3>, <T+4>, <T+5>, <T+10>, <T+20>, <T+30>, <T+40+>




In [17]:
from typing import Literal 
import re

token_mapping = {
    "POS_TOKEN": "<POS>",
    "EGO_POS_TOKEN": "<EGO_POS>",
    "TARGET_POS_TOKEN": "<TRGT>",
    "ORDER_TOKEN": "<ORDER>",
    "INFO_TOKEN": "<INFO>",
    "SENDED_MESSAGE_TOKEN": "<SND>",
    "RECEIVED_MESSAGE_TOKEN": "<RCV>",
    "TO_ME": "<TOME>",
    "AGENT_STATE_TOKEN": "<AGSTATE>",
    "LOCAL_STRATEGY_TOKEN": "<LOCAL_STG>",
    "GLOBAL_STRATEGY_TOKEN": "<GLOBAL_STG>",
    "MISSION_PROGRESS_TOKEN": "<PRGS>",
    "SCOUT_TOKEN": "<SCOUT>",
    "RESCUER_TOKEN": "<RESCUER>",
    "COMMANDER_TOKEN": "<COMMANDER>",
    "ME_FLAG_TOKEN": "<ME>"    
}


def convert_agent_id_to_token_string(agent_id: int) -> str:
    """
    Convert agent ID to a token string.
    """
    return f"<AG#{agent_id}>"

def convert_pos_to_token_string(pos: tuple[int], mode: Literal["ego", "default", "target"] = "ego") -> str:
    """
    Input : (83, 15)
    """
    if mode == "ego":
        return f"{token_mapping['EGO_POS_TOKEN']} {pos[0]} {pos[1]}"
    elif mode == "target":
        return f"{token_mapping['TARGET_POS_TOKEN']} {pos[0]} {pos[1]}"
    else:
        return f"{token_mapping['POS_TOKEN']} {pos[0]} {pos[1]}"


def convert_sended_messages_to_token_string(
    sended_message: list[dict], 
    receiver_id: int,
    message_type: Literal["order", "info"] = "order"
) -> str:
    agent_token = convert_agent_id_to_token_string(receiver_id)
    if message_type == "order":
        message_type_token = token_mapping['ORDER_TOKEN']
    elif message_type == "info":
        message_type_token = token_mapping['INFO_TOKEN']
    else:
        raise ValueError(f"Unknown message type: {message_type}")

    return f"{token_mapping['SENDED_MESSAGE_TOKEN']} {agent_token} {message_type_token} {sended_message}"

def convert_received_messages_to_token_string(
    received_message: list[dict], 
    sender_id: int,
    message_type: Literal["order", "info"] = "order",
    to_me: bool = False
) -> str:
    agent_token = convert_agent_id_to_token_string(sender_id)
    if message_type == "order":
        message_type_token = token_mapping['ORDER_TOKEN']
    elif message_type == "info":
        message_type_token = token_mapping['INFO_TOKEN']
    else:
        raise ValueError(f"Unknown message type: {message_type}")

    to_me_token = token_mapping['TO_ME'] if to_me else ""

    return f"{token_mapping['RECEIVED_MESSAGE_TOKEN']} {agent_token} " + f"{to_me_token} " + f"{message_type_token} {received_message}"


def convert_latest_agent_information_to_token_string(
    agent_id: int, 
    timestamp: int,
    pos: tuple[int, int],
) -> str:
    """
    Convert latest agent information to a token string.
    """
    # TODO : calculate timestamp
    return f"{token_mapping['AGENT_STATE_TOKEN']} {convert_agent_id_to_token_string(agent_id)} {convert_pos_to_token_string(pos, mode='default')}"

def convert_strategy_to_token_string(
    strategy: list[dict], 
    strategy_type: Literal["global", "local"]
) -> str:
    """
    Convert strategy to a token string.
    """
    if strategy_type == "global":
        return f"{token_mapping['GLOBAL_STRATEGY_TOKEN']} {strategy}"
    elif strategy_type == "local":
        return f"{token_mapping['LOCAL_STRATEGY_TOKEN']} {strategy}"
    else:
        raise ValueError(f"Unknown strategy type: {strategy_type}")
    
def convert_mission_progress_to_token_string(
    mission_progress: str
) -> str:
    """
    Convert mission progress to a token string.
    """
    return f"{token_mapping['MISSION_PROGRESS_TOKEN']} {mission_progress}"

def convert_meta_data_to_token_string(agent_id: int, role: Literal["scout", "rescuer", "scout_commander"], mission: str, me_flag: bool) -> str:
    """
    Convert meta data to a token string.
    """
    agent_token = convert_agent_id_to_token_string(agent_id)
    if role.lower() == "scout":
        role_token = token_mapping['SCOUT_TOKEN']
    elif role.lower() == "rescuer":
        role_token = token_mapping['RESCUER_TOKEN']
    elif role.lower() == "scout_commander":
        role_token = token_mapping['COMMANDER_TOKEN']
    else:
        raise ValueError(f"Unknown agent role: {role}")
    
    if me_flag:
        agent_token = f"{token_mapping['ME_FLAG_TOKEN']} {agent_token}"
    
    return f"{agent_token} {role_token} {mission}"

def normalize_agent_references(message: str) -> str:
    
    """
    Example: 
    input:  Scout 10 — redeploy northward. Prioritize systematic visual sweep of northern-central and northeast sectors. 
            Rescuer 2 — maintain current position. Deploy only upon confirmed contact from Scout.
    output: <AG#10> — redeploy northward. Prioritize systematic visual sweep of northern-central and northeast sectors. 
            <AG#2> — maintain current position. Deploy only upon confirmed contact from Scout.'
    """
    message = message.replace("\n", " ")
    message = re.sub(r'\b(Scout|Rescuer)\s+(\d+)\b', r'<AG#\2>', message)
    return message.strip()



In [18]:
msg = """'<RCV> <AG#3>  <ORDER> Scout 10 — redeploy northward. Prioritize systematic visual sweep of northern-central and northeast sectors.
Rescuer 2 — maintain current position. Deploy only upon confirmed contact from Scout.'"""

result = normalize_agent_references(msg)
print(result)


'<RCV> <AG#3>  <ORDER> <AG#10> — redeploy northward. Prioritize systematic visual sweep of northern-central and northeast sectors. <AG#2> — maintain current position. Deploy only upon confirmed contact from Scout.'


In [ ]:
import pandas as pd

sample = train_dataset[1]

def post_process_data_sample(sample):
    meta_info = sample["meta_info"]
    ego_id = sample["ego_id"]
    sample_role = meta_info.loc[meta_info["agent_id"] == ego_id, "role"].values[0]
    
    meta_info_list: list[str] = []
    grand_truth = []
    unique_time_tokens = set()
    
    delete_keys = [
        "id",
        "ego_id",
        "current_timestamp",
        "meta_info"
    ]

    filtered_sample = {k: v for k, v in sample.items() if k not in delete_keys}

    rows = []
    for k, v in filtered_sample.items():
        v_dict: list[dict] = v.to_dict(orient='records')
        for timestamped_item in v_dict:
            timestamp = timestamped_item.pop("timestamp")
            _ = timestamped_item.pop("ego_id", None)
            _ = timestamped_item.pop("id", None)

            rows.append({
                "timestamp": timestamp,
                "data": timestamped_item,
                "key": k,
                "time_delta": "",
                "postprocessed": "",
                "gt": False
            })


    historic_data = pd.DataFrame(rows).sort_values(by="timestamp", ascending=True).reset_index(drop=True)
    max_ts = historic_data['timestamp'].max()
    historic_data.loc[
        (historic_data['timestamp'] == max_ts) &
        (historic_data['key'] == "sended_messages"), # grand truth only for sended messages
        'gt'
        ] = True


    for agent_id, role, mission in zip(meta_info["agent_id"], meta_info["role"], meta_info["mission"]):
        if role == sample_role:
            me_flag = True
        else:
            me_flag = False
        meta_info_list.append(convert_meta_data_to_token_string(agent_id, role, mission, me_flag=me_flag))

    for index, row in historic_data.iterrows():
        timestamp = row["timestamp"]
        prev_timestamp = historic_data["timestamp"].iloc[index - 1] if index > 0 else 0
        data = row["data"]
        key = row["key"]
        gt = row["gt"]
        post_processed = ""
        
        historic_data.at[index, "time_delta"] = timestamp - prev_timestamp

        # Convert position
        if key == "position":
            pos = tuple(data["position"])

        # Convert sended messages
        if key == "sended_messages":
            msg = normalize_agent_references(data["sended_message"])
            receiver_id = data["send_message_to_receiver_id"]
            msg_type = data["sended_message_type"]
            post_processed = convert_sended_messages_to_token_string(
                sended_message=msg,
                receiver_id=receiver_id,
                message_type=msg_type
            )

        # Convert received messages
        if key == "messages_from_agent":
            msg = normalize_agent_references(data["message_from_agent"])
            agent_id = data["message_from_agent_id"]
            msg_type = data["message_from_agent_type"]
            post_processed = convert_received_messages_to_token_string(
                received_message=msg,
                sender_id=agent_id,
                message_type=msg_type,
            )

        # Convert latest agent information
        if key == "latest_agent_information":
            pos_X = data["latest_info_agent_pos_x"]
            pos_Y = data["latest_info_agent_pos_y"]
            agent_id = data["latest_info_agent_id"]
            agent_timestamp = data["latest_info_agent_timestamp"]
            post_processed = convert_latest_agent_information_to_token_string(
                agent_id=agent_id,
                timestamp=agent_timestamp,
                pos=(pos_X, pos_Y)
            )

        # Convert strategies
        if key == "global_strategy":
            global_strategy = data["global_strategy"]
            post_processed = convert_strategy_to_token_string(global_strategy, strategy_type="global")

        if key == "local_strategy" :
            local_strategy = data["local_strategy"]
            post_processed = convert_strategy_to_token_string(local_strategy, strategy_type="local")

        # Convert positions
        if key == "positions":
            pos_X = data["ego_id_pos_x"]
            pos_Y = data["ego_id_pos_y"]
            post_processed = convert_pos_to_token_string((pos_X, pos_Y), mode="ego")
        
        
        # Convert target in FOV
        if key == "target_in_fov":
            target_pos_X = data["target_pos_x"]
            target_pos_Y = data["target_pos_y"]
            post_processed = convert_pos_to_token_string((target_pos_X, target_pos_Y), mode="target")
        
        # Convert mission progress
        if key == "mission_progress":
            mission_progress = data["mission_progress"]
            post_processed = convert_mission_progress_to_token_string(mission_progress)    
        
        if post_processed == "":
            historic_data.at[index, "postprocessed"] = post_processed
        else:
            time_token = f"<T+{historic_data.at[index, 'time_delta']}>"
            unique_time_tokens.add(time_token)
            post_processed_with_time_token = time_token + " " + post_processed
            
            historic_data.at[index, "postprocessed"] = post_processed_with_time_token

        if gt and key == "sended_messages":
            grand_truth.append(post_processed)
    
    
    X = "\n".join(meta_info_list + historic_data[historic_data["gt"] == False]["postprocessed"].to_list())
    y = "\n".join(grand_truth)
    
    return X, y, unique_time_tokens, sample_role, historic_data


X, y, unique_time_tokens, role, historic_data = post_process_data_sample(train_dataset[2])

# print("X:", X)
# print("y:", y)
# print("Unique time tokens:", unique_time_tokens)
# print("Role:", role)

historic_data       

,timestamp,data,key,time_delta,postprocessed,gt
0,0,"{'ego_id_pos_x': 83, 'ego_id_pos_y': 64}",positions,0,<T+0> <EGO_POS> 83 64,False
1,0,{'global_strategy': 'scout targets'},global_strategy,0,<T+0> <GLOBAL_STG> scout targets,False
2,5,"{'target_pos_x': 72, 'target_pos_y': 56}",target_in_fov,5,<T+5> <TRGT> 72 56,False
3,5,"{'ego_id_pos_x': 83, 'ego_id_pos_y': 69}",positions,0,<T+0> <EGO_POS> 83 69,False
4,10,{'mission_progress': 'Recon unit operational. ...,mission_progress,5,<T+5> <PRGS> Recon unit operational. Awaiting ...,False
5,10,"{'send_message_to_receiver_id': 2, 'sended_mes...",sended_messages,0,<T+0> <SND> <AG#2> <INFO> No targets detected ...,False
6,10,"{'message_from_agent_id': 2, 'message_from_age...",messages_from_agent,0,<T+0> <RCV> <AG#2> <INFO> No targets within o...,False
7,10,"{'send_message_to_receiver_id': 3, 'sended_mes...",sended_messages,0,<T+0> <SND> <AG#3> <INFO> No targets detected ...,False
8,10,"{'ego_id_pos_x': 58, 'ego_id_pos_y': 84}",positions,0,<T+0> <EGO_POS> 58 84,False
9,10,"{'target_pos_x': 72, 'target_pos_y': 56}",target_in_fov,0,<T+0> <TRGT> 72 56,False


In [24]:
from tqdm import tqdm

train_unique_time_tokens = set()

scout_train_dataset = []
rescuer_train_dataset = []
commander_train_dataset = []


for sample in tqdm(train_dataset):
    X, y, time_tokens, role, _ = post_process_data_sample(sample)
    train_unique_time_tokens.update(time_tokens)
    
    if role == "scout":
        scout_train_dataset.append((X, y))
    elif role == "rescuer":
        rescuer_train_dataset.append((X, y))
    elif role == "scout_commander":
        commander_train_dataset.append((X, y))
        
        
print(f"Scout dataset : {len(scout_train_dataset)}")
print(f"Rescuer dataset : {len(rescuer_train_dataset)}")
print(f"Commander dataset : {len(commander_train_dataset)}")

100%|██████████| 43/43 [00:00<00:00, 137.66it/s]

Scout dataset : 13
Rescuer dataset : 11
Commander dataset : 19


In [28]:
from tqdm import tqdm

val_unique_time_tokens = set()

scout_val_dataset = []
rescuer_val_dataset = []
commander_val_dataset = []


for sample in tqdm(val_dataset):
    X, y, time_tokens, role, _ = post_process_data_sample(sample)
    val_unique_time_tokens.update(time_tokens)
    
    if role == "scout":
        scout_val_dataset.append((X, y))
    elif role == "rescuer":
        rescuer_val_dataset.append((X, y))
    elif role == "scout_commander":
        commander_val_dataset.append((X, y))
        
        
print(f"Scout dataset : {len(scout_val_dataset)}")
print(f"Rescuer dataset : {len(rescuer_val_dataset)}")
print(f"Commander dataset : {len(commander_val_dataset)}")

100%|██████████| 38/38 [00:00<00:00, 135.01it/s]

Scout dataset : 11
Rescuer dataset : 8
Commander dataset : 19


In [25]:
# print(scout_val_dataset[1][0])
# print("--" * 50)
# print(scout_val_dataset[1][1])

sample_id = 3
print(scout_train_dataset[sample_id][0])
print("--" * 50)
print(scout_train_dataset[sample_id][1])

<ME> <AG#1> <SCOUT> Find the position of targets and send coordinates to rescuer
<AG#2> <RESCUER> Based on the coordinates provided by scout, scan all targets
<AG#3> <COMMANDER> Coordinate scouts and rescuers to find and scan targets
<T+0> <GLOBAL_STG> scout targets
<T+0> <EGO_POS> 83 64
<T+5> <TRGT> 72 56
<T+0> <EGO_POS> 83 69
<T+5> <PRGS> Recon unit operational. Awaiting directive for initial deployment.
<T+0> <SND> <AG#3> <INFO> No targets detected within current visual range.
<T+0> <RCV> <AG#2>  <INFO> No targets within operational range.
<T+0> <SND> <AG#2> <INFO> No targets detected within current visual range.
<T+0> <TRGT> 72 56
<T+0> <AGSTATE> <AG#3> <POS> 11 77
<T+0> <EGO_POS> 58 84
<T+0> <RCV> <AG#3>  <ORDER> <AG#1> — redeploy northward. Prioritize systematic visual sweep of northern-central and northeast sectors. <AG#2> — maintain current position. Deploy only upon confirmed contact from Scout.
<T+0> <AGSTATE> <AG#2> <POS> 62 78
<T+5> <EGO_POS> 58 79
<T+5> <EGO_POS> 58 74
<T+

In [29]:
all_unique_time_tokens = train_unique_time_tokens.union(val_unique_time_tokens)

In [30]:
import pickle

for scout_dataset, filename in zip(
    [scout_train_dataset, scout_val_dataset],
    ["scout_train_dataset.pkl", "scout_val_dataset.pkl"]
):
    with open(filename, 'wb') as f:
        pickle.dump((scout_dataset, all_unique_time_tokens), f)

for rescuer_dataset, filename in zip(
    [rescuer_train_dataset, rescuer_val_dataset],
    ["rescuer_train_dataset.pkl", "rescuer_val_dataset.pkl"]
):
    with open(filename, 'wb') as f:
        pickle.dump((rescuer_dataset, all_unique_time_tokens), f)
        
for commander_dataset, filename in zip(
    [commander_train_dataset, commander_val_dataset],
    ["commander_train_dataset.pkl", "commander_val_dataset.pkl"]
):
    with open(filename, 'wb') as f:
        pickle.dump((commander_dataset, all_unique_time_tokens), f)


In [61]:
with open("scout_train_dataset.pkl", 'rb') as f:
    dataset = pickle.load(f)


dataset_data, unique_time_tokens = dataset

for sample in dataset_data[2:3]:
    X, y = sample
    print("X:", X)
    print("y:", y)
    print("Unique time tokens:", unique_time_tokens)
    print("--" * 50)
    break

X: <ME> <AG#1> <SCOUT> Find the position of targets and send coordinates to rescuer
<AG#2> <RESCUER> Based on the coordinates provided by scout, scan all targets
<AG#3> <COMMANDER> Coordinate scouts and rescuers to find and scan targets
<T+0> <EGO_POS> 83 64
<T+0> <GLOBAL_STG> scout targets
<T+5> <TRGT> 72 56
<T+0> <EGO_POS> 83 69
<T+5> <PRGS> Recon unit operational. Awaiting directive for initial deployment.
<T+0> <SND> <AG#2> <INFO> No targets detected within current visual range.
<T+0> <RCV> <AG#2>  <INFO> No targets within operational range.
<T+0> <SND> <AG#3> <INFO> No targets detected within current visual range.
<T+0> <EGO_POS> 58 84
<T+0> <TRGT> 72 56
<T+0> <RCV> <AG#3>  <ORDER> <AG#1> — redeploy northward. Prioritize systematic visual sweep of northern-central and northeast sectors. <AG#2> — maintain current position. Deploy only upon confirmed contact from Scout.
<T+0> <AGSTATE> <AG#2> <POS> 62 78
<T+0> <AGSTATE> <AG#3> <POS> 11 77
<T+5> <EGO_POS> 58 79
<T+5> <EGO_POS> 58 74


### Convert to Alpaca Dataset

In [31]:
SYSTEM_PROMPT = """You are a multi-agent autonomous coordination model operating in a simulated drone environment. Your role is to analyze past actions, positions, strategies, and messages of all agents and generate the next appropriate communication or order that maintains mission coherence and effectiveness.
Each agent has a designated role:
- <SCOUT>: explores the map and locates targets.
- <RESCUER>: acts upon confirmed targets to scan, secure, or extract.
- <COMMANDER>: issues high-level orders, manages coordination between agents.
Time is marked with <T+X> tokens. Actions occur at or after each time tick. Position tokens like <EGO_POS> or <AGSTATE> indicate coordinates. Strategy, progress, and targets are encoded in specialized tokens.
You must respond using the same structured message language (e.g., <SND> <AG#> <INFO> ...), adhering to all token formats, and include only the most appropriate messages or orders for the current time based on past context.

Agents:
- <AG#1> <SCOUT>: finds targets.
- <AG#2> <RESCUER>: scans or rescues targets using coordinates.
- <AG#3> <COMMANDER>: manages overall mission, coordinates agents.

Message structure:
- <SND>: the agent sends a message.
- <RCV>: the agent receives a message.
- <ORDER>: a directive or command sent to other agents.
- <INFO>: a factual status update (e.g., about targets or sensor state).

Context tokens:
- <T+X>: timestamp, where X is time since mission start.
- <EGO_POS>: the agent’s own coordinates.
- <AGSTATE>: another agent’s known position.
- <TRGT>: known target positions.
- <PRGS>: current mission progress.
- <GLOBAL_STG> / <LOCAL_STG>: high-level or local strategy directives.

"""

INSTRUCTION_PROMPT = """Based on the logs and current context, generate the next agent message or order that continues the mission effectively. The response must follow the standard tokenized message format, such as <SND> <AG#> <INFO> ... or <RCV> <AG#> <ORDER> ... Include only the immediate next action(s).
"""

In [32]:
def convert_sample_into_alpaca_format(x, y, system_prompt, instruction_prompt):
    """
    Convert a sample into the Alpaca format.
    """
    return {
        "instruction": instruction_prompt,
        "input": x,
        "output": y,
        "system": system_prompt
    }

#### Generate dataset for scout in alpaca data format

In [13]:
import pickle

with open("scout_train_dataset.pkl", 'rb') as f:
    scout_train_dataset, scout_unique_time_tokens = pickle.load(f)
    
with open("scout_val_dataset.pkl", 'rb') as f:
    scout_val_dataset, _ = pickle.load(f)

In [33]:
def build_alpaca_dataset(dataset, system_prompt, instruction_prompt):
    alpaca_dataset = []
    for sample in dataset:
        x, y = sample
        alpaca_dataset.append(convert_sample_into_alpaca_format(x, y, system_prompt, instruction_prompt))
    return alpaca_dataset

scout_train_alpaca_dataset = build_alpaca_dataset(scout_train_dataset, SYSTEM_PROMPT, INSTRUCTION_PROMPT)
scout_val_alpaca_dataset = build_alpaca_dataset(scout_val_dataset, SYSTEM_PROMPT, INSTRUCTION_PROMPT)

rescuer_train_alpaca_dataset = build_alpaca_dataset(rescuer_train_dataset, SYSTEM_PROMPT, INSTRUCTION_PROMPT)
rescuer_val_alpaca_dataset = build_alpaca_dataset(rescuer_val_dataset, SYSTEM_PROMPT, INSTRUCTION_PROMPT)

commander_train_alpaca_dataset = build_alpaca_dataset(commander_train_dataset, SYSTEM_PROMPT, INSTRUCTION_PROMPT)
commander_val_alpaca_dataset = build_alpaca_dataset(commander_val_dataset, SYSTEM_PROMPT, INSTRUCTION_PROMPT)

In [ ]:
import json
from pathlib import Path
import pickle


def save_alpaca_dataset_to_json(dataset: list[dict], path: Path):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)

def save_special_tokens_to_pkl_file(special_tokens: list[str], path: Path):
    with open(path, 'wb') as f:
        pickle.dump(special_tokens, f)
    

alpaca_dataset_path = Path("./alpaca_datasets")
alpaca_dataset_path.mkdir(parents=True, exist_ok=True)

save_alpaca_dataset_to_json(scout_train_alpaca_dataset, alpaca_dataset_path / "scout_train_alpaca_dataset.json")
save_alpaca_dataset_to_json(scout_val_alpaca_dataset, alpaca_dataset_path / "scout_val_alpaca_dataset.json")

save_alpaca_dataset_to_json(rescuer_train_alpaca_dataset, alpaca_dataset_path / "rescuer_train_alpaca_dataset.json")
save_alpaca_dataset_to_json(rescuer_val_alpaca_dataset, alpaca_dataset_path / "rescuer_val_alpaca_dataset.json")

save_alpaca_dataset_to_json(commander_train_alpaca_dataset, alpaca_dataset_path / "commander_train_alpaca_dataset.json")
save_alpaca_dataset_to_json(commander_val_alpaca_dataset, alpaca_dataset_path / "commander_val_alpaca_dataset.json")

save_special_tokens_to_pkl_file(list(scout_unique_time_tokens) + list(token_mapping.values()), alpaca_dataset_path / "special_tokens.pkl")

In [ ]:
#   "drone_rescue_small": {
#     "file_name": "scout_train_alpaca_dataset.json",
#     "columns": {
#       "prompt": "instruction",
#       "query": "input",
#       "response": "output",
#       "system": "system"
#     }
#   },
#   "drone_rescue_val_small": {
#     "file_name": "scout_val_alpaca_dataset.json",
#     "columns": {
#       "prompt": "instruction",
#       "query": "input",
#       "response": "output",
#       "system": "system"
#     }
#   }

In [ ]:
{
    "predict_bleu-4": 24.10579999999999,
    "predict_model_preparation_time": 0.0028,
    "predict_rouge-1": 39.56391818181818,
    "predict_rouge-2": 29.485563636363636,
    "predict_rouge-l": 33.92617272727273,
    "predict_runtime": 88.8039,
    "predict_samples_per_second": 0.124,
    "predict_steps_per_second": 0.068
}